In [ ]:
# General setup for all examples
import foolbox as fb
import torchvision.models as models

# instantiate a model
model = models.resnet18(pretrained=True).eval()
preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
fmodel = fb.models.PyTorchModel(model, bounds=(0, 1), preprocessing=preprocessing)

# get data and test the model
images, labels = fb.utils.samples(fmodel, dataset='imagenet', batchsize=4)
print(fb.utils.accuracy(fmodel, images, labels))
# -> 1.0

### Minimum example to use Brendel & Bethge attack

In [ ]:
# apply the Brendel & Bethge attack
attack = fb.attacks.L2BrendelBethgeAttack(lr=1, steps=100)
adversarials, _, adv_mask = attack(
    fmodel,
    images,
    criterion=fb.criteria.Misclassification(labels),
    epsilons=1
)
print(fb.utils.accuracy(fmodel, adversarials, labels))
# -> 0.0

### Using Brendel & Bethge with starting points selected from data set (to ensure valid starting points)

In [ ]:
# get all data
images, labels = fb.utils.samples(fmodel, dataset='imagenet', batchsize=20)

# split images and labels into 'batches' (just to mirror the batches of standard large-scale datasets)
batches = [(images[:10], labels[:10]), (images[10:], labels[10:])]

# create attack that picks adversarials from given dataset of samples
init_attack = fb.attacks.DatasetAttack()

init_attack.feed(fmodel, batches[0][0])   # feed 1st batch of inputs
init_attack.feed(fmodel, batches[1][0])   # feed 2nd batch of inputs

# apply the Brendel & Bethge attack
attack = fb.attacks.L2BrendelBethgeAttack(init_attack=init_attack, steps=100)
adversarials, _, adv_mask = attack(
    fmodel,
    images,
    criterion=fb.criteria.Misclassification(labels),
    epsilons=1
)
print(fb.utils.accuracy(fmodel, adversarials, labels))
# -> 0.0